In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import re
from sklearn.feature_extraction.text import CountVectorizer
import wrds
import duckdb
db = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [2]:
data = pd.read_pickle('./data/con_arousal_happy_20230803.pkl', compression='zip')

In [3]:
data_agg = pd.DataFrame()
data_agg['transcript'] = data.groupby(['eventid', 'conversationid'])['transcript'].apply(lambda x: ''.join(x))
data_agg['transcript'] = data_agg['transcript'].apply(lambda x: x.lower())
data['ret'] = np.where(data['trading_hours'] == 1, data['trade_ret'], data['q_ret'])
data_agg['ret'] = data.groupby(['eventid', 'conversationid'])['ret'].apply(lambda x: x.iloc[0])
data_agg['ticker'] = data.groupby(['eventid', 'conversationid'])['ticker2'].apply(lambda x: x.iloc[0])
data_agg['starttime'] = data.groupby(['eventid', 'conversationid'])['con_starttime2'].apply(lambda x: x.iloc[0])
data_agg['endtime'] = data.groupby(['eventid', 'conversationid'])['con_endtime2'].apply(lambda x: x.iloc[0])
data_agg['starttime'] = pd.to_datetime(data_agg['starttime'])
data_agg['endtime'] = pd.to_datetime(data_agg['endtime'])
data_agg['date'] = data_agg.endtime.dt.date
data_agg['transcript'] = data_agg['transcript'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))
data_agg['transcript'] = data_agg['transcript'].apply(lambda x: re.sub('\s+', ' ', x))
data_agg = data_agg.reset_index()

In [4]:
stocknames = db.get_table('crsp', 'stocknames')

In [5]:
data_agg = duckdb.query("""
SELECT a.*, b.*
FROM data_agg as a
LEFT JOIN stocknames as b
ON a.ticker = b.ticker and a.date <= b.nameenddt and a.date >= b.namedt
""").df().drop_duplicates()
data_agg = data_agg.loc[~data_agg['shrcls'].isin(['B', 'V', 'P'])]
data_agg = data_agg[['eventid', 'conversationid', 'transcript',
                     'ret', 'ticker', 'starttime', 'endtime',
                     'date', 'permno', 'cusip']].dropna()

In [6]:
ccmxpf = db.raw_sql("""
SELECT *
FROM crsp.ccmxpf_linktable
WHERE LINKTYPE IN ('LU', 'LC', 'LD', 'LF', 'LN', 'LO', 'LS', 'LX')
""")
ccmxpf['linkenddt'] = np.where(ccmxpf['linkenddt'].isna(), '2023-12-31', ccmxpf['linkenddt'])

In [7]:
data_agg = duckdb.query("""
SELECT a.*, b.gvkey
FROM data_agg as a
LEFT JOIN ccmxpf as b
ON a.permno = b.lpermno and a.date >= linkdt and a.date <= linkenddt
""").df().drop_duplicates()
data_agg = data_agg.loc[~(data_agg['gvkey'] == '011550')]
data_agg = data_agg.dropna().drop_duplicates()

In [10]:
data_agg['callDate'] = data_agg['date']
data_agg['fyearq'] = data_agg.date.dt.year
def last_day_of_previous_quarter(date):
    quarter = date.quarter
    year = date.year
    if quarter == 1:
        return dt.date(year - 1, 12, 31)
    elif quarter == 2:
        return dt.date(year, 3, 31)
    elif quarter == 3:
        return dt.date(year, 6, 30)
    elif quarter == 4:
        return dt.date(year, 9, 30)
data_agg['datadate'] = data_agg['callDate'].apply(lambda x: last_day_of_previous_quarter(x))

In [11]:
data_agg

,eventid,conversationid,transcript,ret,ticker,starttime,endtime,date,permno,cusip,gvkey,callDate,fyearq,datadate
0,010320_LW_137951422184,1,two questions if i could tom i guess first as ...,0.004060,LW,2020-01-03 10:27:15,2020-01-03 10:31:15,2020-01-03,16431.0,51327210,028790,2020-01-03,2020,2019-12-31
1,010320_LW_137951422184,2,i guess i was hoping maybe you could go on the...,0.002623,LW,2020-01-03 10:31:25,2020-01-03 10:35:26,2020-01-03,16431.0,51327210,028790,2020-01-03,2020,2019-12-31
2,010320_LW_137951422184,3,you gave a lot of helpful color on the potato ...,-0.000327,LW,2020-01-03 10:35:39,2020-01-03 10:37:57,2020-01-03,16431.0,51327210,028790,2020-01-03,2020,2019-12-31
3,010320_LW_137951422184,4,i just had a question for you first on a comme...,0.003156,LW,2020-01-03 10:38:05,2020-01-03 10:40:58,2020-01-03,16431.0,51327210,028790,2020-01-03,2020,2019-12-31
4,010320_LW_137951422184,5,so i just i m really impressed with this volum...,-0.001520,LW,2020-01-03 10:41:06,2020-01-03 10:43:34,2020-01-03,16431.0,51327210,028790,2020-01-03,2020,2019-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77751,110818_MRO_137822779557,5,i guess just following up on some of the capit...,0.016712,MRO,2018-11-08 09:37:05,2018-11-08 09:43:16,2018-11-08,15069.0,56584910,007017,2018-11-08,2018,2018-09-30
77752,110818_MRO_137822779557,6,mitch you covered most of the questions i had ...,-0.007953,MRO,2018-11-08 09:43:27,2018-11-08 09:46:44,2018-11-08,15069.0,56584910,007017,2018-11-08,2018,2018-09-30
77753,110818_MRO_137822779557,7,lee you highlighted when you were talking abou...,-0.006400,MRO,2018-11-08 09:46:53,2018-11-08 09:51:09,2018-11-08,15069.0,56584910,007017,2018-11-08,2018,2018-09-30
77754,110818_MRO_137822779557,8,what i think would be helpful and i think you ...,0.002685,MRO,2018-11-08 09:51:24,2018-11-08 09:56:44,2018-11-08,15069.0,56584910,007017,2018-11-08,2018,2018-09-30


In [ ]:
data_agg.to_csv('./data/con_data.csv')

In [13]:
data_event = pd.DataFrame()
data_event['transcript'] = data_agg.groupby(['eventid'])['transcript'].apply(lambda x: ''.join(x))
data_event['transcript'] = data_event['transcript'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))
data_event['transcript'] = data_event['transcript'].apply(lambda x: re.sub('\s+', ' ', x))
data_event['ticker'] = data_agg.groupby(['eventid'])['ticker'].apply(lambda x: x.iloc[0])
data_event['starttime'] = data_agg.groupby(['eventid'])['starttime'].apply(lambda x: x.iloc[0])
data_event['endtime'] = data_agg.groupby(['eventid'])['endtime'].apply(lambda x: x.iloc[-1])
data_event['starttime'] = pd.to_datetime(data_event['starttime'])
data_event['endtime'] = pd.to_datetime(data_event['endtime'])
data_event['date'] = data_event.endtime.dt.date
data_event['date'] = pd.to_datetime(data_event['date'])
data_event['permno'] = data_agg.groupby(['eventid'])['permno'].apply(lambda x: x.iloc[0])
data_event['cusip'] = data_agg.groupby(['eventid'])['cusip'].apply(lambda x: x.iloc[0])
data_event['gvkey'] = data_agg.groupby(['eventid'])['gvkey'].apply(lambda x: x.iloc[0])
data_event['callDate'] = data_event['date']
data_event['fyearq'] = data_event.date.dt.year
data_event['datadate'] = data_event['callDate'].apply(lambda x: last_day_of_previous_quarter(x))

In [16]:
data_event.to_csv('./data/event_data.csv')

In [59]:
meta = data_agg[['permno', 'gvkey', 'datadate', 'fyearq', 'callDate', 'ret']]
meta.to_csv('./data/con_meta.csv')

In [60]:
text = data_agg['transcript'].reset_index(drop=True)
vectorizer = CountVectorizer()
dtm_matrix = vectorizer.fit_transform(text)
dtm_df = pd.DataFrame(dtm_matrix.toarray(), columns=vectorizer.get_feature_names_out())

In [63]:
dtm_df.to_csv('./data/con_dtm.csv')

KeyboardInterrupt: 